In [176]:
print('sss')

sss


In [177]:
import pandas as pd
import numpy as np

In [178]:
## read the file
path=r'C:\Users\arvinmvn\Desktop\Ranking\Ranking_IN_Sample_Output3.xlsx'
df=pd.read_excel(path)

df.head()

,provider_id,service_name
0,4f796148-38fc-441d-857e-c8db05ca6309,Excess Inventory
1,901aa6b4-267f-483b-b6b1-bb6a606452ea,Storage
2,3b1b5637-fe6d-4b3a-bed8-915a6662862d,Accounting
3,ef573518-fc0c-4f61-82d3-6898d0f73484,Advertising Optimization
4,0ff82c94-f00a-4ce3-ad97-90525a5a2d23,Seller Reinstatement


We will create a view that has provider id, service name,Standard Metrics 1,Standard Metrics 2,Standard Metrics 3,Standard Metrics 4,Standard Metrics 5,Standard Metrics 6,Service Metrics 1,Service Metrics 2

after that we will assign the weights and calculate scores by creating blocks for each service

lets take individual sheets 1st, each sheet has provider id vs service vs metric, we will take it in different data frames and go a join on provider_id and service

In [179]:
provider_list=pd.read_excel(path,sheet_name='provider_list')

In [180]:
ytc_percent_in_week=pd.read_excel(path,sheet_name='YTC % (In week)')
#ytc_percent_in_week=ytc_percent_in_week[['providerid','service','ytc_in_week_percent']]

In [181]:
ytc_sla_ltd_21_7_days=pd.read_excel(path,sheet_name='YTC SLA (LTD 21-7 days)')
ytc_sla_ltd_21_7_days=ytc_sla_ltd_21_7_days[['providerid','service','ytc_sla_21_7']]

In [182]:
ytc_percent_ltd_21_7_days=pd.read_excel(path,sheet_name='YTC % (LTD 21 - 7 days)')
ytc_percent_ltd_21_7_days=ytc_percent_ltd_21_7_days[ytc_percent_ltd_21_7_days['marketplaceid'].notnull()]
ytc_percent_ltd_21_7_days=ytc_percent_ltd_21_7_days[['providerid','service','ytc_ltd_21_7_percent']]

In [183]:
snr_orders_completed=pd.read_excel(path,sheet_name='snr_orders(completed)')
snr_orders_completed['service']='Seller Reinstatement'

In [184]:
cat_orders_completed=pd.read_excel(path,sheet_name='cat_orders(completed)')
cat_orders_completed['service']='Cataloguing'

In [185]:
orders_completed=pd.read_excel(path,sheet_name='Orders(completed)')

## Add SNR orders

## we will replace the services keywords from orders query so that join is mapped correctly, we will also add snr into this

orders_completed=orders_completed.replace( ['Advertise','Boost','FBA','Enhanced Brand Content'], ['Advertising Optimization','Account Management','FBA Preparation','Enhanced Brand Content'] )

orders_completed=pd.concat([orders_completed[['providerid','service','total_4w_orders']],
           snr_orders_completed[snr_orders_completed['metric']=='No. of orders'][['providerid','service','orders']].rename(columns = {'orders': 'total_4w_orders'}, inplace = False)
          ])

## add calalog orders
orders_completed=pd.concat([orders_completed,cat_orders_completed.rename(columns = {'no_orders': 'total_4w_orders'}, inplace = False)])

In [186]:
avg_feedback=pd.read_excel(path,sheet_name='avg_feedback')

In [187]:
unique_seller=pd.read_excel(path,sheet_name='unique_seller')
unique_seller=unique_seller.replace( ['Advertise','Boost','FBA','Enhanced Brand Content'], ['Advertising Optimization','Account Management','FBA Preparation','Enhanced Brand Content'] )
unique_seller=pd.concat([unique_seller,snr_orders_completed[snr_orders_completed['metric']=='No. of sellers'][['providerid','service','orders']].rename(columns = {'orders': 'adoption'}, inplace = False)
          ])        

In [188]:
pip_percent=pd.read_excel(path,sheet_name='pip_%')

pip_percent=pip_percent[['providerid','service','pip_percent']]

In [189]:
caq=pd.read_excel(path,sheet_name='CAQ')
caq['service']='Cataloguing'

In [190]:
cat_asins_created=pd.read_excel(path,sheet_name='cat_asins_created')
cat_asins_created['service']='Cataloguing'

In [191]:
ad_revenue=pd.read_excel(path,sheet_name='Ad Revenue')
ad_revenue['service']='Advertising Optimization'

#ad_revenue=ad_revenue.groupby(['providerid','service'])['sp_revenue','attributed_gms','budget','gms','ad_revenue','acos_p'].sum().reset_index()
ad_revenue=ad_revenue[['providerid','total_spend','service']]

In [192]:
## we are the 4 weeks of acos data and doing an average of them

## we are doing reset index to ensure that the df is not getting converted to a series.
ad_acos=pd.read_excel(path,sheet_name='ad_ACOS')
ad_acos['service']='Advertising Optimization'
#ad_acos=ad_acos.groupby(['providerid','service'])['sp_revenue','attributed_gms','budget','gms','ad_revenue','acos_p'].sum().reset_index()

ad_acos['acos_p']=ad_acos['numeraor']/ad_acos['denominator']
ad_acos['acos_p']=ad_acos['acos_p'].replace(np.inf,0)
ad_acos = ad_acos.groupby(['providerid','service'])['acos_p'].mean().reset_index()
ad_acos=ad_acos[['providerid','acos_p','service']]


In [193]:
AM_gms=pd.read_excel(path,sheet_name='AM_gms')
AM_gms['service']='Account Management'
AM_gms=AM_gms.rename(columns = {'gms': 'boost_gms'}, inplace = False)

In [194]:
ebc_gv=pd.read_excel(path,sheet_name='ebc_gv')
ebc_gv['service']='Enhanced Brand Content'
ebc_gv=ebc_gv.groupby(['provider_id','service'])['asins_created','lw_gv_platform','lw_gv_3p','lw_gv_3p_total','lw_gv_platform_total','spn_flag','lw_gv_3p_ytd','lw_gv_platform_ytd','lw_gv_3p_total','lw_gv_platform_total'].sum().reset_index()
ebc_gv=ebc_gv.rename(columns = {'asins_created': 'ebc_gv_asins_created'}, inplace = False)

C:\Users\arvinmvn\AppData\Local\Temp\ipykernel_16672\820171744.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ebc_gv=ebc_gv.groupby(['provider_id','service'])['asins_created','lw_gv_platform','lw_gv_3p','lw_gv_3p_total','lw_gv_platform_total','spn_flag','lw_gv_3p_ytd','lw_gv_platform_ytd','lw_gv_3p_total','lw_gv_platform_total'].sum().reset_index()


In [195]:
ebc_asin_coverage=pd.read_excel(path,sheet_name='ebc_asin_coverage')
ebc_asin_coverage['service']='Enhanced Brand Content'
ebc_asin_coverage=ebc_asin_coverage.groupby(['provider_id','service'])['asins_created'].sum().reset_index()
ebc_asin_coverage=ebc_asin_coverage.rename(columns = {'asins_created': 'ebc_asin_coverage_asins_created'}, inplace = False)

In [196]:
tax_unblocked_ttm=pd.read_excel(path,sheet_name='tax_unblocked_ttm')
tax_unblocked_ttm['service']='Taxes'
tax_unblocked_ttm=tax_unblocked_ttm.rename(columns = {'unblocked_gms': 'tax_unblocked_gms'}, inplace = False)

In [197]:
SNR_reinstatement_p_num=pd.read_excel(path,sheet_name='SNR_reinstatement_p_num')

In [198]:
SNR_reinstatement_p_deno=pd.read_excel(path,sheet_name='SNR_reinstatement_p_deno')

In [199]:

## filter out numerator and denomenator from the respective blocks and do a join on provider id vs num vs deno
snr_reinstatement_p=pd.merge(
    SNR_reinstatement_p_num[['providerid','no_sellers']].rename(columns = {'no_sellers': 'snr_reinst_p_num'}, inplace = False),
    SNR_reinstatement_p_deno[['providerid','ytd']].rename(columns = {'ytd': 'snr_reinst_p_deno'}, inplace = False),
    how='outer', 
    left_on =['providerid'],
    right_on=['providerid']
    )
## create a view on providerid vs (num/deno) and fill na with o
snr_reinstatement_p['snr_reinst_p']=snr_reinstatement_p['snr_reinst_p_num']/snr_reinstatement_p['snr_reinst_p_deno']
snr_reinstatement_p=snr_reinstatement_p.fillna(0).round(4)
snr_reinstatement_p['service']='Seller Reinstatement'
snr_reinstatement_p=snr_reinstatement_p[['providerid','snr_reinst_p','service']]

In [200]:
ds_ncns=pd.read_excel(path,sheet_name='DS_NCNS')
ds_ncns['service']='Domestic Shipping'

In [201]:
compliance_unblocked_gms=pd.read_excel(path,sheet_name='compliance_unblocked_gms')
compliance_unblocked_gms['service']='Compliance'
compliance_unblocked_gms=compliance_unblocked_gms.rename(columns = {'unblocked_gms': 'compliance_unblocked_gms'}, inplace = False)

In [202]:
join_1_2=pd.merge(
    provider_list,
    ytc_percent_in_week.rename(columns = {'providerid': 'providerid_1', 'service': 'service_1'}, inplace = False),
    how='left',
    left_on=['provider_id','service_name'],
    right_on=['providerid_1','service_1']
    )

In [203]:
len(join_1_2)

6027

In [204]:
join_1_2.columns

Index(['provider_id', 'service_name', 'providerid_1', 'service_1',
       'ytc_in_week_numerator', 'ytc_in_week_denomenator',
       'ytc_in_week_percent'],
      dtype='object')

In [205]:
#ytc_sla_ltd_21_7_days.rename(columns = {'providerid': 'providerid_1', 'service': 'service_1'}, inplace = False)

In [206]:
join_1_3=pd.merge(
    join_1_2.drop(['providerid_1','service_1'],axis=1, inplace=False),
    ytc_sla_ltd_21_7_days.rename(columns = {'providerid': 'providerid_2', 'service': 'service_2'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_2','service_2']
    )

In [207]:
len(join_1_3)

6027

In [208]:
join_1_3.columns

Index(['provider_id', 'service_name', 'ytc_in_week_numerator',
       'ytc_in_week_denomenator', 'ytc_in_week_percent', 'providerid_2',
       'service_2', 'ytc_sla_21_7'],
      dtype='object')

In [209]:
join_1_4=pd.merge(
    join_1_3.drop(['providerid_2','service_2'],axis=1, inplace=False),
    ytc_percent_ltd_21_7_days.rename(columns = {'providerid': 'providerid_3', 'service': 'service_3'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_3','service_3']
    )

In [210]:
join_1_4.to_clipboard()

In [211]:
len(join_1_4)

6027

In [212]:
join_1_5=pd.merge(
    join_1_4.drop(['providerid_3','service_3'],axis=1, inplace=False),
    orders_completed.rename(columns = {'providerid': 'providerid_4', 'service': 'service_4'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_4','service_4']
    )

In [213]:
len(join_1_5)

6027

In [214]:
join_1_6=pd.merge(
    join_1_5.drop(['providerid_4','service_4'],axis=1, inplace=False),
    cat_orders_completed.rename(columns = {'providerid': 'providerid_5', 'service': 'service_5'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_5','service_5']
    )

In [215]:
len(join_1_6)

6027

In [216]:
snr_reinstatement_p.columns

Index(['providerid', 'snr_reinst_p', 'service'], dtype='object')

In [217]:
join_1_7=pd.merge(
    join_1_6.drop(['providerid_5','service_5'],axis=1, inplace=False),
    snr_reinstatement_p.rename(columns = {'providerid': 'providerid_6', 'service': 'service_6'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_6','service_6']
    )

In [218]:
len(join_1_7)

6027

In [219]:
join_1_8=pd.merge(
    join_1_7.drop(['providerid_6','service_6'],axis=1, inplace=False),
    avg_feedback.rename(columns = {'provider_id': 'providerid_7', 'services_applicable': 'service_7'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_7','service_7']
    )

In [220]:
len(join_1_8)

6027

In [221]:
join_1_9=pd.merge(
    join_1_8.drop(['providerid_7','service_7'],axis=1, inplace=False),
    unique_seller.rename(columns = {'providerid': 'providerid_8', 'service': 'service_8'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_8','service_8']
    )

In [222]:
len(join_1_9)

6027

In [223]:
join_1_10=pd.merge(
    join_1_9.drop(['providerid_8','service_8'],axis=1, inplace=False),
    pip_percent.rename(columns = {'providerid': 'providerid_9', 'service': 'service_9'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_9','service_9']
    )

In [224]:
len(join_1_10)

6027

In [225]:
join_1_11=pd.merge(
    join_1_10.drop(['providerid_9','service_9'],axis=1, inplace=False),
    caq.rename(columns = {'providerid': 'providerid_10', 'service': 'service_10'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_10','service_10']
    )

In [226]:
len(join_1_11)

6027

In [227]:
join_1_12=pd.merge(
    join_1_11.drop(['providerid_10','service_10'],axis=1, inplace=False),
    cat_asins_created.rename(columns = {'provider_name': 'providerid_11', 'service': 'service_11'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_11','service_11']
    )

In [228]:
len(join_1_12)

6027

In [229]:
join_1_13=pd.merge(
    join_1_12.drop(['providerid_11','service_11'],axis=1, inplace=False),
    ad_revenue.rename(columns = {'providerid': 'providerid_12', 'service': 'service_12'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_12','service_12']
    )

In [230]:
len(join_1_13)

6027

In [231]:
join_1_14=pd.merge(
    join_1_13.drop(['providerid_12','service_12'],axis=1, inplace=False),
    ad_acos.rename(columns = {'providerid': 'providerid_13', 'service': 'service_13'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_13','service_13']
    )

In [232]:
len(join_1_14)

6027

In [233]:
join_1_15=pd.merge(
    join_1_14.drop(['providerid_13','service_13'],axis=1, inplace=False),
    AM_gms.rename(columns = {'providerid': 'providerid_14', 'service': 'service_14'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_14','service_14']
    )

In [234]:
len(join_1_15)

6027

In [235]:
join_1_16=pd.merge(
    join_1_15.drop(['providerid_14','service_14'],axis=1, inplace=False),
    ebc_gv.rename(columns = {'provider_id': 'providerid_15', 'service': 'service_15'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_15','service_15']
    )

In [236]:
len(join_1_16)

6027

In [237]:
join_1_17=pd.merge(
    join_1_16.drop(['providerid_15','service_15'],axis=1, inplace=False),
    ebc_asin_coverage.rename(columns = {'provider_id': 'providerid_16', 'service': 'service_16'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_16','service_16']
    )

In [238]:
len(join_1_17)

6027

In [239]:
join_1_18=pd.merge(
    join_1_17.drop(['providerid_16','service_16'],axis=1, inplace=False),
    tax_unblocked_ttm.rename(columns = {'providerid': 'providerid_17', 'service': 'service_17'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_17','service_17']
    )

In [240]:
len(join_1_18)

6027

In [241]:
join_1_19=pd.merge(
    join_1_18.drop(['providerid_17','service_17'],axis=1, inplace=False),
    ds_ncns.rename(columns = {'provider_id': 'providerid_18', 'service': 'service_18'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_18','service_18']
    )

In [242]:
len(join_1_19)

6027

In [243]:
join_1_20=pd.merge(
    join_1_19.drop(['providerid_18','service_18'],axis=1, inplace=False),
    compliance_unblocked_gms.rename(columns = {'providerid': 'providerid_19', 'service': 'service_19'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_19','service_19']
    )

In [244]:
len(join_1_20)

6027

In [245]:
join_1_20.to_clipboard()

In [246]:
join_1_20.columns

Index(['provider_id', 'service_name', 'ytc_in_week_numerator',
       'ytc_in_week_denomenator', 'ytc_in_week_percent', 'ytc_sla_21_7',
       'ytc_ltd_21_7_percent', 'total_4w_orders', 'no_orders', 'snr_reinst_p',
       'avg_feedback', 'adoption', 'pip_percent', 'caq_score',
       'count_of_asins', 'total_spend', 'acos_p', 'boost_gms',
       'ebc_gv_asins_created', 'lw_gv_platform', 'lw_gv_3p', 'lw_gv_3p_total',
       'lw_gv_platform_total', 'spn_flag', 'lw_gv_3p_ytd',
       'lw_gv_platform_ytd', 'lw_gv_3p_total', 'lw_gv_platform_total',
       'ebc_asin_coverage_asins_created', 'tax_unblocked_gms', 'ncns',
       'providerid_19', 'unblocked_asin', 'compliance_unblocked_gms',
       'service_19'],
      dtype='object')

In [247]:
## we have the metrics view prepared, few niggles that need to be checked are
# provider id list for cataloging for those which has that spn codes

In [248]:
## Ranking view for imaging

In [249]:
## let's have the weights for imaging in this block

ytc_percent_in_week_percent_imaging_wt= 10/100
ytc_sla_ltd_21_7_days_imaging_wt= 5/100
ytc_percent_ltd_21_7_days_imaging_wt= 5/100
pip_percent_imaging_wt= 10/100
orders_completed_imaging_wt= 35/100
unique_seller_imaging_wt= 15/100
avg_feedback_imaging_wt= 20/100


In [250]:
join_1_20.columns

Index(['provider_id', 'service_name', 'ytc_in_week_numerator',
       'ytc_in_week_denomenator', 'ytc_in_week_percent', 'ytc_sla_21_7',
       'ytc_ltd_21_7_percent', 'total_4w_orders', 'no_orders', 'snr_reinst_p',
       'avg_feedback', 'adoption', 'pip_percent', 'caq_score',
       'count_of_asins', 'total_spend', 'acos_p', 'boost_gms',
       'ebc_gv_asins_created', 'lw_gv_platform', 'lw_gv_3p', 'lw_gv_3p_total',
       'lw_gv_platform_total', 'spn_flag', 'lw_gv_3p_ytd',
       'lw_gv_platform_ytd', 'lw_gv_3p_total', 'lw_gv_platform_total',
       'ebc_asin_coverage_asins_created', 'tax_unblocked_gms', 'ncns',
       'providerid_19', 'unblocked_asin', 'compliance_unblocked_gms',
       'service_19'],
      dtype='object')

In [251]:
## imaging scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

imag_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

imag_rank=imag_rank[imag_rank['service_name']=='Imaging']
imag_rank=imag_rank.fillna(0)

##prepare the score

imag_rank['score']=(  ytc_percent_in_week_percent_imaging_wt*imag_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_imaging_wt*imag_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_imaging_wt*imag_rank['ytc_ltd_21_7_percent']
                    + pip_percent_imaging_wt*imag_rank['pip_percent']
                    + orders_completed_imaging_wt*imag_rank['total_4w_orders']
                    + unique_seller_imaging_wt*imag_rank['adoption']
                    + avg_feedback_imaging_wt*imag_rank['avg_feedback']
                   )

imag_rank['rank']=imag_rank[['score','total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
imag_rank = imag_rank.sort_values(by="rank", ignore_index=True)
imag_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,160779a5-e457-4a57-bb42-f14bbc8ea4e4,Imaging,0.00,0.76,0.0,47.83,2077.0,75.0,5.00,744.021,1.0
1,a5c32bce-5fe4-4f64-8764-7987f12eab79,Imaging,0.18,0.00,0.0,44.44,1227.0,0.0,4.94,434.900,2.0
2,10fddeed-85f8-4f4f-9f5c-b7ead466b0b1,Imaging,0.00,0.00,0.0,65.22,972.0,1.0,5.00,347.872,3.0
3,7ef4588b-b4f9-4ec2-b162-fa820c1a8236,Imaging,0.00,0.00,0.0,30.00,806.0,9.0,4.82,287.414,4.0
4,860468c6-9e94-4627-8e89-23e58e944fff,Imaging,1.20,0.00,0.0,43.33,181.0,859.0,4.90,197.633,5.0


In [252]:
## let's have the weights for Accounting in this block

ytc_percent_in_week_percent_accounting_wt= 20/100
ytc_sla_ltd_21_7_days_accounting_wt= 5/100
ytc_percent_ltd_21_7_days_accounting_wt= 5/100
#pip_percent_accounting_wt= 10/100
orders_completed_accounting_wt= 30/100
unique_seller_accounting_wt= 20/100
avg_feedback_accounting_wt= 20/100

In [253]:
## Accounting scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

accounting_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

accounting_rank=accounting_rank[accounting_rank['service_name']=='Accounting']
accounting_rank=accounting_rank.fillna(0)

##prepare the score

accounting_rank['score']=(  ytc_percent_in_week_percent_accounting_wt*accounting_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_accounting_wt*accounting_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_accounting_wt*accounting_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_accounting_wt*accounting_rank['pip_percent']
                    + orders_completed_accounting_wt*accounting_rank['total_4w_orders']
                    + unique_seller_accounting_wt*accounting_rank['adoption']
                    + avg_feedback_accounting_wt*accounting_rank['avg_feedback']
                   )

accounting_rank['rank']=accounting_rank[['score','total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
accounting_rank = accounting_rank.sort_values(by="rank", ignore_index=True)
accounting_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,c9f3c2fb-0628-485b-8308-4a03c8699434,Accounting,0.0,0.0,0.0,36.84,71.0,67.0,4.92,35.684,1.0
1,825a388d-26d6-40a6-8e11-49e536698664,Accounting,100.0,0.0,100.0,0.00,0.0,0.0,1.00,25.200,2.0
2,7efb2627-714b-45d2-b7cb-9308ad5ef017,Accounting,100.0,50.0,50.0,100.00,0.0,0.0,0.00,25.000,3.0
3,75fa340c-4b59-4fb7-a6b0-f1985db60a68,Accounting,100.0,0.0,100.0,0.00,0.0,0.0,0.00,25.000,3.0
4,43a1a46d-59be-41f9-b093-204afa9883cd,Accounting,100.0,0.0,0.0,100.00,4.0,1.0,5.00,22.400,4.0


In [254]:
## let's have the weights for Training in this block

ytc_percent_in_week_percent_training_wt= 20/100
ytc_sla_ltd_21_7_days_training_wt= 5/100
ytc_percent_ltd_21_7_days_training_wt= 5/100
#pip_percent_training_wt= 10/100
orders_completed_training_wt= 30/100
unique_seller_training_wt= 20/100
avg_feedback_training_wt= 20/100

In [255]:
##Training scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

training_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

training_rank=training_rank[training_rank['service_name']=='Training']
training_rank=training_rank.fillna(0)

##prepare the score

training_rank['score']=(  ytc_percent_in_week_percent_training_wt*training_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_training_wt*training_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_training_wt*training_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_training_wt*training_rank['pip_percent']
                    + orders_completed_training_wt*training_rank['total_4w_orders']
                    + unique_seller_training_wt*training_rank['adoption']
                    + avg_feedback_training_wt*training_rank['avg_feedback']
                   )

training_rank['rank']=training_rank[['score','total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
training_rank = training_rank.sort_values(by="rank", ignore_index=True)
training_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,7ef4588b-b4f9-4ec2-b162-fa820c1a8236,Training,0.00,0.00,0.0,38.71,838.0,0.0,4.68,252.336,1.0
1,218919a7-661e-4e1c-a721-26be982ef3e7,Training,6.96,0.00,0.0,51.06,430.0,1.0,4.50,131.492,2.0
2,26ddd595-c3a2-413f-bec2-2de54264ede8,Training,10.60,5.74,0.0,46.43,382.0,0.0,5.00,118.007,3.0
3,d7c02675-303c-498f-b9f7-105fe9eccdea,Training,0.00,0.00,0.0,0.00,385.0,0.0,4.60,116.420,4.0
4,10fddeed-85f8-4f4f-9f5c-b7ead466b0b1,Training,0.00,0.00,0.0,66.67,171.0,0.0,0.00,51.300,5.0


In [256]:
## let's have the weights for International Shipping in this block

ytc_percent_in_week_percent_i_s_wt= 20/100
ytc_sla_ltd_21_7_days_i_s_wt= 5/100
ytc_percent_ltd_21_7_days_i_s_wt= 5/100
#pip_percent_i_s_wt= 10/100
orders_completed_i_s_wt= 30/100
unique_seller_i_s_wt= 20/100
avg_feedback_i_s_wt= 20/100

In [257]:
##International Shipping scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

i_s_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

i_s_rank=i_s_rank[i_s_rank['service_name']=='International Shipping']
i_s_rank=i_s_rank.fillna(0)

##prepare the score

i_s_rank['score']=(  ytc_percent_in_week_percent_i_s_wt*i_s_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_i_s_wt*i_s_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_i_s_wt*i_s_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_i_s_wt*i_s_rank['pip_percent']
                    + orders_completed_i_s_wt*i_s_rank['total_4w_orders']
                    + unique_seller_i_s_wt*i_s_rank['adoption']
                    + avg_feedback_i_s_wt*i_s_rank['avg_feedback']
                   )

i_s_rank['rank']=i_s_rank[['score','total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
i_s_rank = i_s_rank.sort_values(by="rank", ignore_index=True)
i_s_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,f28c3e53-bdb8-452c-8892-30c888af8991,International Shipping,100.0,75.0,25.0,40.0,0.0,0.0,0.0,25.0,1.0
1,6a9c04cb-4c34-4fa6-8d4a-b65561aeec1c,International Shipping,100.0,0.0,100.0,0.0,0.0,0.0,0.0,25.0,1.0
2,b3b3e999-8cc7-4f13-b1ff-7bd4ce4a0648,International Shipping,100.0,0.0,100.0,0.0,0.0,0.0,0.0,25.0,1.0
3,44fa0c95-d4c5-4109-9a95-df0c028bd71b,International Shipping,100.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,2.0
4,aee9b211-16e1-4f24-b609-d8f36dd2cb68,International Shipping,100.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,2.0


In [258]:
## let's have the weights for Translation in this block

ytc_percent_in_week_percent_trans_wt= 20/100
ytc_sla_ltd_21_7_days_trans_wt= 5/100
ytc_percent_ltd_21_7_days_trans_wt= 5/100
#pip_percent_trans_wt= 10/100
orders_completed_trans_wt= 30/100
unique_seller_trans_wt= 20/100
avg_feedback_trans_wt= 20/100

In [259]:
##Translation scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

trans_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

trans_rank=trans_rank[trans_rank['service_name']=='Translation']
trans_rank=trans_rank.fillna(0)

##prepare the score

trans_rank['score']=(  ytc_percent_in_week_percent_trans_wt*trans_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_trans_wt*trans_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_trans_wt*trans_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_i_s_wt*trans_rank['pip_percent']
                    + orders_completed_trans_wt*trans_rank['total_4w_orders']
                    + unique_seller_trans_wt*trans_rank['adoption']
                    + avg_feedback_trans_wt*trans_rank['avg_feedback']
                   )

trans_rank['rank']=trans_rank[['score','total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
trans_rank = trans_rank.sort_values(by="rank", ignore_index=True)
trans_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,71fcbc79-acd4-4ace-82cc-cd8ca682ecb5,Translation,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.8,1.0
1,17b3aaab-2ebc-4a99-b6d1-642d36f7b0e4,Translation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,acf33907-2e82-41af-b23b-abb74976ec47,Translation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,e96e6f84-8d07-4d48-89fe-40a36bb3f0d2,Translation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,5cbda68b-9b12-4b83-a044-a10a92baa99c,Translation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [260]:
## let's have the weights for International Returns in this block

ytc_percent_in_week_percent_i_r_wt= 20/100
ytc_sla_ltd_21_7_days_i_r_wt= 5/100
ytc_percent_ltd_21_7_days_i_r_wt= 5/100
#pip_percent_i_r_wt= 10/100
orders_completed_i_r_wt= 30/100
unique_seller_i_r_wt= 20/100
avg_feedback_i_r_wt= 20/100

In [261]:
##International Shipping scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

i_r_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

i_r_rank=i_r_rank[i_r_rank['service_name']=='International Returns']
i_r_rank=i_r_rank.fillna(0)

##prepare the score

i_r_rank['score']=(  ytc_percent_in_week_percent_i_r_wt*i_r_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_i_r_wt*i_r_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_i_r_wt*i_r_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_i_r_wt*i_r_rank['pip_percent']
                    + orders_completed_i_r_wt*i_r_rank['total_4w_orders']
                    + unique_seller_i_r_wt*i_r_rank['adoption']
                    + avg_feedback_i_r_wt*i_r_rank['avg_feedback']
                   )

i_r_rank['rank']=i_r_rank[['score','total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
i_r_rank = i_r_rank.sort_values(by="rank", ignore_index=True)
i_r_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,f28c3e53-bdb8-452c-8892-30c888af8991,International Returns,100.0,0.0,100.0,0.0,0.0,0.0,0.0,25.0,1.0
1,ce11ae06-6774-49d5-abc4-70a805b7826e,International Returns,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,b33f6d0b-2fd6-4fc5-afbe-96865e5e5cf7,International Returns,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,e07d88c6-9ecb-4991-a27f-aa3f04ca2814,International Returns,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,71291d76-43f5-4b91-bdcf-713080294d85,International Returns,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [262]:
## let's have the weights for FBA in this block

ytc_percent_in_week_percent_fba_wt= 20/100
ytc_sla_ltd_21_7_days_fba_wt= 5/100
ytc_percent_ltd_21_7_days_fba_wt= 5/100
#pip_percent_fba_wt= 10/100
orders_completed_fba_wt= 30/100
unique_seller_fba_wt= 20/100
avg_feedback_fba_wt= 20/100

In [263]:
## FBA scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

fba_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

fba_rank=fba_rank[fba_rank['service_name']=='FBA Preparation']
fba_rank=fba_rank.fillna(0)

##prepare the score

fba_rank['score']=(  ytc_percent_in_week_percent_fba_wt*fba_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_fba_wt*fba_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_fba_wt*fba_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_fba_wt*fba_rank['pip_percent']
                    + orders_completed_fba_wt*fba_rank['total_4w_orders']
                    + unique_seller_fba_wt*fba_rank['adoption']
                    + avg_feedback_fba_wt*fba_rank['avg_feedback']
                   )

fba_rank['rank']=fba_rank[['score','total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
fba_rank = fba_rank.sort_values(by="rank", ignore_index=True)
fba_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,504878ee-ff2f-46b3-98f7-77d5e7ed139b,FBA Preparation,0.0,0.0,0.0,22.22,47.0,25.0,5.0,20.1,1.0
1,d2f317ff-cbd0-4846-9b47-18fe8273c6ae,FBA Preparation,0.0,0.0,0.0,0.00,18.0,10.0,5.0,8.4,2.0
2,9024e1be-44b7-4d2e-9d2d-6db10e8c0ded,FBA Preparation,0.0,0.0,0.0,0.00,14.0,14.0,5.0,8.0,3.0
3,a021786e-3c21-48be-955a-549382f3a3ae,FBA Preparation,0.0,0.0,0.0,0.00,13.0,13.0,5.0,7.5,4.0
4,76893e47-0687-4638-a77d-acc82d61f561,FBA Preparation,0.0,0.0,100.0,0.00,0.0,0.0,0.0,5.0,5.0


In [264]:
## let's have the weights for storage in this block

ytc_percent_in_week_percent_stor_wt= 20/100
ytc_sla_ltd_21_7_days_stor_wt= 5/100
ytc_percent_ltd_21_7_days_stor_wt= 5/100
#pip_percent_stor_wt= 10/100
orders_completed_stor_wt= 30/100
unique_seller_stor_wt= 20/100
avg_feedback_stor_wt= 20/100

In [265]:
## Storage scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

stor_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

stor_rank=stor_rank[stor_rank['service_name']=='Storage']
stor_rank=stor_rank.fillna(0)

##prepare the score

stor_rank['score']=(  ytc_percent_in_week_percent_stor_wt*stor_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_stor_wt*stor_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_stor_wt*stor_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_stor_wt*stor_rank['pip_percent']
                    + orders_completed_stor_wt*stor_rank['total_4w_orders']
                    + unique_seller_stor_wt*stor_rank['adoption']
                    + avg_feedback_stor_wt*stor_rank['avg_feedback']
                   )

stor_rank['rank']=stor_rank[['score','total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
stor_rank = stor_rank.sort_values(by="rank", ignore_index=True)
stor_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,ad3442f1-f0c5-4a84-b3c3-64a61fe4dbe7,Storage,10.34,0.00,0.00,0.00,120.0,111.0,5.0,61.2680,1.0
1,fb45d4ec-2467-4035-a153-93ec5331d753,Storage,2.27,0.00,0.00,0.00,110.0,107.0,1.0,55.0540,2.0
2,bbef9f87-0261-4212-9b6e-60922e19c662,Storage,100.00,7.14,35.71,0.00,39.0,29.0,5.0,40.6425,3.0
3,c9f3c2fb-0628-485b-8308-4a03c8699434,Storage,0.00,0.00,0.00,64.29,63.0,63.0,5.0,32.5000,4.0
4,72ea85b8-ee37-4db6-92ff-506b63b4de97,Storage,100.00,0.00,100.00,0.00,0.0,0.0,0.0,25.0000,5.0


In [266]:
## let's have the weights for IPA in this block

ytc_percent_in_week_percent_ipa_wt= 20/100
ytc_sla_ltd_21_7_days_ipa_wt= 5/100
ytc_percent_ltd_21_7_days_ipa_wt= 5/100
#pip_percent_ipa_wt= 10/100
orders_completed_ipa_wt= 30/100
unique_seller_ipa_wt= 20/100
avg_feedback_ipa_wt= 20/100

In [267]:
## IPA scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

ipa_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

ipa_rank=ipa_rank[ipa_rank['service_name']=='IP Accelerator']
ipa_rank=ipa_rank.fillna(0)

##prepare the score

ipa_rank['score']=(  ytc_percent_in_week_percent_ipa_wt*ipa_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_ipa_wt*ipa_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_ipa_wt*ipa_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_ipa_wt*ipa_rank['pip_percent']
                    + orders_completed_ipa_wt*ipa_rank['total_4w_orders']
                    + unique_seller_ipa_wt*ipa_rank['adoption']
                    + avg_feedback_ipa_wt*ipa_rank['avg_feedback']
                   )

ipa_rank['rank']=ipa_rank[['score','total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
ipa_rank = ipa_rank.sort_values(by="rank", ignore_index=True)
ipa_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,6a9c04cb-4c34-4fa6-8d4a-b65561aeec1c,IP Accelerator,100.00,0.00,100.0,0.00,0.0,0.0,0.0,25.0000,1.0
1,bef07376-64e0-4df9-b5c0-8ba8c135ea04,IP Accelerator,100.00,40.00,40.0,80.00,0.0,0.0,0.0,24.0000,2.0
2,685b69a4-690a-4e60-b312-da59f31c46a0,IP Accelerator,50.00,66.67,0.0,0.00,0.0,0.0,0.0,13.3335,3.0
3,c50388ea-f913-4cf1-aa47-7d9fb8ca8aaf,IP Accelerator,50.00,42.86,0.0,44.44,0.0,0.0,0.0,12.1430,4.0
4,eb02c90d-fc6c-438d-8c3d-fa33de18fe34,IP Accelerator,21.88,37.50,0.0,54.29,0.0,0.0,0.0,6.2510,5.0


In [268]:
## let's have the weights for Excess Inventory  in this block

ytc_percent_in_week_percent_exec_wt= 20/100
ytc_sla_ltd_21_7_days_exec_wt= 5/100
ytc_percent_ltd_21_7_days_exec_wt= 5/100
#pip_percent_exec_wt= 10/100
orders_completed_exec_wt= 30/100
unique_seller_exec_wt= 20/100
avg_feedback_exec_wt= 20/100

In [269]:
## Excesss Inventory scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

exec_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

exec_rank=exec_rank[exec_rank['service_name']=='Excess Inventory']
exec_rank=exec_rank.fillna(0)

##prepare the score

exec_rank['score']=(  ytc_percent_in_week_percent_exec_wt*exec_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_exec_wt*exec_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_exec_wt*exec_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_exec_wt*exec_rank['pip_percent']
                    + orders_completed_exec_wt*exec_rank['total_4w_orders']
                    + unique_seller_exec_wt*exec_rank['adoption']
                    + avg_feedback_exec_wt*exec_rank['avg_feedback']
                   )

exec_rank['rank']=exec_rank[['score','total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
exec_rank = exec_rank.sort_values(by="rank", ignore_index=True)
exec_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,0a8e300b-68a1-42b5-9ad1-c953710dde00,Excess Inventory,100.0,0.0,100.00,0.0,0.0,0.0,0.0,25.0000,1.0
1,5a3bdad8-0f7b-45be-bade-d2e8b13d468f,Excess Inventory,100.0,0.0,100.00,0.0,0.0,0.0,0.0,25.0000,1.0
2,b6209911-111c-448f-b152-4d1ccd8a91b9,Excess Inventory,100.0,0.0,100.00,0.0,0.0,0.0,0.0,25.0000,1.0
3,bc3c2364-16f3-43b9-b775-17a921b27459,Excess Inventory,100.0,0.0,100.00,0.0,0.0,0.0,0.0,25.0000,1.0
4,c0d52ba3-3eb7-457e-a8f0-02bfe947ea24,Excess Inventory,100.0,0.0,66.67,100.0,0.0,0.0,0.0,23.3335,2.0


In [270]:
## let's have the weights for Cataloging in this block

ytc_percent_in_week_percent_cat_wt= 10/100
ytc_sla_ltd_21_7_days_cat_wt= 10/100
ytc_percent_ltd_21_7_days_cat_wt= 10/100
#pip_percent_cat_wt= 10/100
orders_completed_cat_wt= 20/100
unique_seller_cat_wt= 10/100
avg_feedback_cat_wt= 10/100
caq_cat_wt=20/100
cat_asin_created_wt=10/100

In [271]:
## Excesss Catalouging scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'caq_score',
       'count_of_asins']

cat_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

cat_rank=cat_rank[cat_rank['service_name']=='Cataloguing']
cat_rank=cat_rank.fillna(0)

##prepare the score

cat_rank['score']=(  ytc_percent_in_week_percent_cat_wt*cat_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_cat_wt*cat_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_cat_wt*cat_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_cat_wt*cat_rank['pip_percent']
                    + orders_completed_cat_wt*cat_rank['total_4w_orders']
                    + unique_seller_cat_wt*cat_rank['adoption']
                    + avg_feedback_cat_wt*cat_rank['avg_feedback']
                    + caq_cat_wt*cat_rank['caq_score']
                    + cat_asin_created_wt*cat_rank['count_of_asins']
                   )

cat_rank['rank']=cat_rank[['score','total_4w_orders','caq_score','avg_feedback','adoption','ytc_in_week_percent','count_of_asins']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
cat_rank = cat_rank.sort_values(by="rank", ignore_index=True)
cat_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,caq_score,count_of_asins,score,rank
0,b3f2f78a-5443-48ea-8b17-28b337100ccc,Cataloguing,0.00,0.49,0.0,72.40,682.0,0.0,5.00,0.74,1685091.0,168646.197,1.0
1,160779a5-e457-4a57-bb42-f14bbc8ea4e4,Cataloguing,0.28,1.92,0.0,30.97,462.0,0.0,4.47,0.79,346212.0,34714.425,2.0
2,10fddeed-85f8-4f4f-9f5c-b7ead466b0b1,Cataloguing,0.00,0.00,0.0,41.34,193.0,0.0,5.00,0.85,332674.0,33306.670,3.0
3,61e20b59-c64b-4548-851c-faac0c37a434,Cataloguing,0.00,0.00,0.0,0.00,135.0,0.0,5.00,0.92,233236.0,23351.284,4.0
4,860468c6-9e94-4627-8e89-23e58e944fff,Cataloguing,0.00,0.00,0.0,42.61,504.0,0.0,4.81,0.83,196000.0,19701.447,5.0


In [272]:
## let's have the weights for taxes in this block

ytc_percent_in_week_percent_tax_wt= 20/100
ytc_sla_ltd_21_7_days_tax_wt= 5/100
ytc_percent_ltd_21_7_days_tax_wt= 5/100
#pip_percent_tax_wt= 10/100
orders_completed_tax_wt= 30/100
unique_seller_tax_wt= 20/100
avg_feedback_tax_wt= 20/100

In [273]:
## Excesss Inventory scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'tax_unblocked_gms']

tax_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

tax_rank=tax_rank[tax_rank['service_name']=='Taxes']
tax_rank=tax_rank.fillna(0)

##prepare the score

tax_rank['score']=(  ytc_percent_in_week_percent_tax_wt*tax_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_tax_wt*tax_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_tax_wt*tax_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_tax_wt*tax_rank['pip_percent']
                    + orders_completed_tax_wt*tax_rank['total_4w_orders']
                    + unique_seller_tax_wt*tax_rank['adoption']
                    + avg_feedback_tax_wt*tax_rank['avg_feedback']
                   )

tax_rank['rank']=tax_rank[['score','total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
tax_rank = tax_rank.sort_values(by="rank", ignore_index=True)
tax_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,tax_unblocked_gms,score,rank
0,160779a5-e457-4a57-bb42-f14bbc8ea4e4,Taxes,0.00,0.0,0.0,62.50,732.0,530.0,5.0,1.630258e+06,326.60,1.0
1,10fddeed-85f8-4f4f-9f5c-b7ead466b0b1,Taxes,1.05,0.0,0.0,43.75,430.0,424.0,0.0,5.768702e+05,214.01,2.0
2,8ab05dea-5450-42ed-b06a-b3e49a8e1f05,Taxes,0.00,0.0,0.0,100.00,139.0,56.0,0.0,2.976150e+07,52.90,3.0
3,c9f3c2fb-0628-485b-8308-4a03c8699434,Taxes,0.00,0.0,0.0,64.29,114.0,58.0,5.0,3.078227e+08,46.80,4.0
4,377b05e0-08ad-4a05-bd7b-50f4b0379abc,Taxes,0.00,0.0,0.0,33.33,59.0,49.0,5.0,7.658944e+08,28.50,5.0


In [274]:
## let's have the weights for compliance in this block

ytc_percent_in_week_percent_comp_wt= 20/100
ytc_sla_ltd_21_7_days_comp_wt= 5/100
ytc_percent_ltd_21_7_days_comp_wt= 5/100
#pip_percent_comp_wt= 10/100
orders_completed_comp_wt= 30/100
unique_seller_comp_wt= 20/100
avg_feedback_comp_wt= 20/100

In [275]:
## Excesss Compliance scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'compliance_unblocked_gms',
        'unblocked_asin']

comp_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

comp_rank=comp_rank[comp_rank['service_name']=='Compliance']
comp_rank=comp_rank.fillna(0)

##prepare the score

comp_rank['score']=(  ytc_percent_in_week_percent_comp_wt*comp_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_comp_wt*comp_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_comp_wt*comp_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_comp_wt*comp_rank['pip_percent']
                    + orders_completed_comp_wt*comp_rank['total_4w_orders']
                    + unique_seller_comp_wt*comp_rank['adoption']
                    + avg_feedback_comp_wt*comp_rank['avg_feedback']
                   )

comp_rank['rank']=comp_rank[['score','total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
comp_rank = comp_rank.sort_values(by="rank", ignore_index=True)
comp_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,compliance_unblocked_gms,unblocked_asin,score,rank
0,10fddeed-85f8-4f4f-9f5c-b7ead466b0b1,Compliance,0.0,0.0,0.0,36.36,196.0,83.0,0.0,0.00,0.0,75.4,1.0
1,8ab05dea-5450-42ed-b06a-b3e49a8e1f05,Compliance,0.0,0.0,0.0,0.00,139.0,56.0,0.0,221645.97,24.0,52.9,2.0
2,00bb70b7-b526-430d-8979-584b329b6746,Compliance,100.0,0.0,100.0,0.00,0.0,0.0,0.0,0.00,0.0,25.0,3.0
3,6a9c04cb-4c34-4fa6-8d4a-b65561aeec1c,Compliance,100.0,0.0,100.0,0.00,0.0,0.0,0.0,0.00,0.0,25.0,3.0
4,6340a578-2a34-4e9f-800a-ffb14e6ac0ac,Compliance,100.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,20.0,4.0


In [276]:
## for adz account management ebc and DS, we have ytc my understanding is it is ytc to contact numerator.this we have to check 

In [277]:
## let's have the weights for adz in this block

ytc_percent_in_week_num_adz_wt= 20/100
orders_completed_adz_wt= 20/100
unique_seller_adz_wt= 15/100
avg_feedback_adz_wt= 15/100
acos_adz_wt=10/100
ad_revenue_adz_wt=25/100

In [278]:
join_1_20.columns

Index(['provider_id', 'service_name', 'ytc_in_week_numerator',
       'ytc_in_week_denomenator', 'ytc_in_week_percent', 'ytc_sla_21_7',
       'ytc_ltd_21_7_percent', 'total_4w_orders', 'no_orders', 'snr_reinst_p',
       'avg_feedback', 'adoption', 'pip_percent', 'caq_score',
       'count_of_asins', 'total_spend', 'acos_p', 'boost_gms',
       'ebc_gv_asins_created', 'lw_gv_platform', 'lw_gv_3p', 'lw_gv_3p_total',
       'lw_gv_platform_total', 'spn_flag', 'lw_gv_3p_ytd',
       'lw_gv_platform_ytd', 'lw_gv_3p_total', 'lw_gv_platform_total',
       'ebc_asin_coverage_asins_created', 'tax_unblocked_gms', 'ncns',
       'providerid_19', 'unblocked_asin', 'compliance_unblocked_gms',
       'service_19'],
      dtype='object')

In [279]:
## Ad Optimization scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_numerator',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'total_spend',
        'acos_p']

adz_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

adz_rank=adz_rank[adz_rank['service_name']=='Advertising Optimization']
adz_rank=adz_rank.fillna(0)

##prepare the score

adz_rank['score']=( ytc_percent_in_week_num_adz_wt*adz_rank['ytc_in_week_numerator']
                    + orders_completed_adz_wt*adz_rank['total_4w_orders']
                    + unique_seller_adz_wt*adz_rank['adoption']
                    + avg_feedback_adz_wt*adz_rank['avg_feedback']
                    + ad_revenue_adz_wt*adz_rank['total_spend']
                    + acos_adz_wt*adz_rank['acos_p']
                   )
adz_rank['rank']=adz_rank[['score','total_spend','total_4w_orders','avg_feedback','adoption','ytc_in_week_numerator']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
adz_rank = adz_rank.sort_values(by="rank", ignore_index=True)
adz_rank.head()

,provider_id,service_name,ytc_in_week_numerator,total_4w_orders,adoption,avg_feedback,total_spend,acos_p,score,rank
0,860468c6-9e94-4627-8e89-23e58e944fff,Advertising Optimization,0.0,600.0,31.0,4.56,1.157713e+08,0.217226,2.894296e+07,1.0
1,ac1398cb-72ca-443c-8ae6-8f764e1297f4,Advertising Optimization,0.0,3.0,0.0,5.00,5.859970e+07,0.244733,1.464993e+07,2.0
2,8efd01b7-d87b-4ba9-acf1-172256164c85,Advertising Optimization,0.0,6.0,0.0,5.00,5.112100e+07,0.411462,1.278025e+07,3.0
3,a021786e-3c21-48be-955a-549382f3a3ae,Advertising Optimization,0.0,11.0,0.0,4.80,3.643153e+07,0.201585,9.107885e+06,4.0
4,2a64ecda-082d-4361-b5c4-2cad8740c59d,Advertising Optimization,0.0,38.0,11.0,5.00,2.229053e+07,0.187910,5.572643e+06,5.0


In [281]:
## let's have the weights for account management in this block

ytc_percent_in_week_num_am_wt= 20/100
orders_completed_am_wt= 20/100
unique_seller_am_wt= 15/100
avg_feedback_am_wt= 15/100
am_gms_wt=30/100

In [282]:
## AM scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_numerator',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'boost_gms']

am_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

am_rank=am_rank[am_rank['service_name']=='Account Management']
am_rank=am_rank.fillna(0)

##prepare the score

am_rank['score']=( ytc_percent_in_week_num_am_wt*am_rank['ytc_in_week_numerator']
                    + orders_completed_am_wt*am_rank['total_4w_orders']
                    + unique_seller_am_wt*am_rank['adoption']
                    + avg_feedback_am_wt*am_rank['avg_feedback']
                    + am_gms_wt*am_rank['boost_gms']
                   )

am_rank['rank']=am_rank[['score','boost_gms','total_4w_orders','avg_feedback','adoption','ytc_in_week_numerator']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
am_rank = am_rank.sort_values(by="rank", ignore_index=True)
am_rank.head()

,provider_id,service_name,ytc_in_week_numerator,total_4w_orders,adoption,avg_feedback,boost_gms,score,rank
0,b975ba43-4856-42de-8857-7d3551e647b3,Account Management,1.0,100.0,0.0,4.55,7.614578e+08,2.284373e+08,1.0
1,04407379-d775-4a69-9d63-bf5b5e01b282,Account Management,2.0,109.0,1.0,4.96,1.363627e+08,4.090883e+07,2.0
2,90fb720f-19d1-4bab-b1ab-c918355b7b4d,Account Management,0.0,9.0,0.0,4.55,9.043392e+07,2.713018e+07,3.0
3,860468c6-9e94-4627-8e89-23e58e944fff,Account Management,3.0,226.0,4.0,4.79,7.281577e+07,2.184478e+07,4.0
4,a021786e-3c21-48be-955a-549382f3a3ae,Account Management,0.0,5.0,0.0,4.89,6.046527e+07,1.813958e+07,5.0


In [283]:
## let's have the weights for EBC in this block

ytc_percent_in_week_num_ebc_wt= 15/100
orders_completed_ebc_wt= 20/100
unique_seller_ebc_wt= 15/100
avg_feedback_ebc_wt= 15/100
ebc_gv_wt=25/100
ebc_asin_coverage_wt=10/100

In [284]:
## EBC scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_numerator',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'ebc_gv_asins_created',
        'lw_gv_platform']

ebc_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

ebc_rank=ebc_rank[ebc_rank['service_name']=='Enhanced Brand Content']
ebc_rank=ebc_rank.fillna(0)

##prepare the score

ebc_rank['score']=( ytc_percent_in_week_num_ebc_wt*ebc_rank['ytc_in_week_numerator']
                    + orders_completed_ebc_wt*ebc_rank['total_4w_orders']
                    + unique_seller_ebc_wt*ebc_rank['adoption']
                    + avg_feedback_ebc_wt*ebc_rank['avg_feedback']
                    + ebc_gv_wt*ebc_rank['lw_gv_platform']
                    + ebc_asin_coverage_wt*ebc_rank['ebc_gv_asins_created']
                   )

ebc_rank['rank']=ebc_rank[['score','lw_gv_platform','total_4w_orders','avg_feedback','adoption','ytc_in_week_numerator']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
ebc_rank = ebc_rank.sort_values(by="rank", ignore_index=True)
ebc_rank.head()

,provider_id,service_name,ytc_in_week_numerator,total_4w_orders,adoption,avg_feedback,ebc_gv_asins_created,lw_gv_platform,score,rank
0,860468c6-9e94-4627-8e89-23e58e944fff,Enhanced Brand Content,0.0,0.0,0.0,4.48,17169.0,2330284.0,584288.5720,1.0
1,a3edbea2-0b75-494c-80b9-d8fa529f2b31,Enhanced Brand Content,1.0,0.0,0.0,5.00,15260.0,2147188.0,538323.9000,2.0
2,e624d8bb-038f-4e19-97bf-0f949769944b,Enhanced Brand Content,0.0,0.0,0.0,0.00,4158.0,1825205.0,456717.0500,3.0
3,f6ab326d-bcd5-4f98-bd54-3c058782f049,Enhanced Brand Content,0.0,0.0,0.0,4.75,2975.0,1786724.0,446979.2125,4.0
4,95665c31-4bd2-4325-ac87-67df9363f705,Enhanced Brand Content,0.0,0.0,0.0,0.00,898.0,1112347.0,278176.5500,5.0


In [285]:
## let's have the weights for SNR in this block

ytc_percent_in_week_num_snr_wt= 20/100
#pip_percent_snr_wt= 10/100
orders_completed_snr_wt= 20/100
unique_seller_snr_wt= 15/100
avg_feedback_snr_wt= 15/100
snr_reinstatement_p_wt=30/100

In [286]:
## SNR scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_numerator',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'snr_reinst_p']

snr_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

snr_rank=snr_rank[snr_rank['service_name']=='Seller Reinstatement']
snr_rank=snr_rank.fillna(0)

##prepare the score

snr_rank['score']=( ytc_percent_in_week_num_snr_wt*snr_rank['ytc_in_week_numerator']
                    + orders_completed_snr_wt*snr_rank['total_4w_orders']
                    + unique_seller_snr_wt*snr_rank['adoption']
                    + avg_feedback_snr_wt*snr_rank['avg_feedback']
                    + snr_reinstatement_p_wt*snr_rank['snr_reinst_p']
                   )

snr_rank['rank']=snr_rank[['score','snr_reinst_p','total_4w_orders','avg_feedback','adoption','ytc_in_week_numerator']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
snr_rank = snr_rank.sort_values(by="rank", ignore_index=True)
snr_rank.head()

,provider_id,service_name,ytc_in_week_numerator,total_4w_orders,adoption,avg_feedback,snr_reinst_p,score,rank
0,88c9ed93-d745-47db-95b7-1cc8acb1192b,Seller Reinstatement,0.0,1043.0,1016.0,0.00,0.0017,361.00051,1.0
1,90fb720f-19d1-4bab-b1ab-c918355b7b4d,Seller Reinstatement,1.0,800.0,791.0,3.66,0.0151,279.40353,2.0
2,04407379-d775-4a69-9d63-bf5b5e01b282,Seller Reinstatement,0.0,165.0,122.0,5.00,0.2873,52.13619,3.0
3,c9f3c2fb-0628-485b-8308-4a03c8699434,Seller Reinstatement,2.0,17.0,15.0,5.00,0.0000,6.80000,4.0
4,504878ee-ff2f-46b3-98f7-77d5e7ed139b,Seller Reinstatement,0.0,21.0,8.0,5.00,0.0000,6.15000,5.0


In [287]:
## let's have the weights for DS in this block

ytc_percent_in_week_num_ds_wt= 20/100
#pip_percent_snr_wt= 10/100
orders_completed_ds_wt= 20/100
unique_seller_ds_wt= 15/100
avg_feedback_ds_wt= 15/100
ds_ncns_p_wt=30/100

In [288]:
## DS scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_numerator',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'ncns']

ds_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

ds_rank=ds_rank[ds_rank['service_name']=='Domestic Shipping']
ds_rank=ds_rank.fillna(0)

##prepare the score

ds_rank['score']=( ytc_percent_in_week_num_ds_wt*ds_rank['ytc_in_week_numerator']
                    + orders_completed_ds_wt*ds_rank['total_4w_orders']
                    + unique_seller_ds_wt*ds_rank['adoption']
                    + avg_feedback_ds_wt*ds_rank['avg_feedback']
                    + ds_ncns_p_wt*ds_rank['ncns']
                   )

ds_rank['rank']=ds_rank[['score','ncns','total_4w_orders','avg_feedback','adoption','ytc_in_week_numerator']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
ds_rank = ds_rank.sort_values(by="rank", ignore_index=True)
ds_rank.head()

,provider_id,service_name,ytc_in_week_numerator,total_4w_orders,adoption,avg_feedback,ncns,score,rank
0,186b1aa4-24a8-454a-8aea-d4517546408e,Domestic Shipping,12.0,0.0,0.0,0.0,0.0,2.4,1.0
1,9aec9e35-135b-417e-af34-ce19772b8edd,Domestic Shipping,8.0,0.0,0.0,0.0,0.0,1.6,2.0
2,9c9912f9-76bc-4e16-9396-ae01ea4518bf,Domestic Shipping,4.0,0.0,0.0,0.0,0.0,0.8,3.0
3,6a9c04cb-4c34-4fa6-8d4a-b65561aeec1c,Domestic Shipping,4.0,0.0,0.0,0.0,0.0,0.8,3.0
4,57df9332-9ed3-4266-8408-097fb7e21941,Domestic Shipping,3.0,0.0,0.0,0.0,0.0,0.6,4.0


In [289]:
## concatenate all the services and rank in 1 view

final_rank_sss= pd.concat([
    imag_rank[['provider_id','service_name','rank']],
    accounting_rank[['provider_id','service_name','rank']],
    training_rank[['provider_id','service_name','rank']],
    i_s_rank[['provider_id','service_name','rank']],
    trans_rank[['provider_id','service_name','rank']],
    i_r_rank[['provider_id','service_name','rank']],
    fba_rank[['provider_id','service_name','rank']],
    stor_rank[['provider_id','service_name','rank']],
    ipa_rank[['provider_id','service_name','rank']],
    exec_rank[['provider_id','service_name','rank']],
    cat_rank[['provider_id','service_name','rank']],
    tax_rank[['provider_id','service_name','rank']],
    comp_rank[['provider_id','service_name','rank']],
    adz_rank[['provider_id','service_name','rank']],
    am_rank[['provider_id','service_name','rank']],
    ebc_rank[['provider_id','service_name','rank']],
    snr_rank[['provider_id','service_name','rank']],
    ds_rank[['provider_id','service_name','rank']]

    ],ignore_index=True)

final_rank_sss.head()

,provider_id,service_name,rank
0,160779a5-e457-4a57-bb42-f14bbc8ea4e4,Imaging,1.0
1,a5c32bce-5fe4-4f64-8764-7987f12eab79,Imaging,2.0
2,10fddeed-85f8-4f4f-9f5c-b7ead466b0b1,Imaging,3.0
3,7ef4588b-b4f9-4ec2-b162-fa820c1a8236,Imaging,4.0
4,860468c6-9e94-4627-8e89-23e58e944fff,Imaging,5.0


In [290]:
final_rank_sss.to_clipboard()

In [291]:
## have to check out on caq empty and the proviider ids for cat
## NCNS is giving empty results have to check that
## we can use the new ad rev query and take a provider level cut and same we can do for boost gms
## have to use dense ranking instead of normal ranking

In [292]:
## consolidate all the data frames into 1 single report
path= r'C:\Users\arvinmvn\Desktop\Ranking\ranking_in_output_python.xlsx'

writer = pd.ExcelWriter(path,engine='xlsxwriter')
# Write each dataframe to a different worksheet.
final_rank_sss.to_excel(writer, sheet_name='final_rank',header = True,index=False)
join_1_20.to_excel(writer, sheet_name='metrics_raw',header = True,index=False)
imag_rank.to_excel(writer, sheet_name='imaging',header = True,index=False)
accounting_rank.to_excel(writer, sheet_name='accounting',header = True,index=False)
training_rank.to_excel(writer, sheet_name='training',header = True,index=False)
i_s_rank.to_excel(writer, sheet_name='international_shipping',header = True,index=False)
trans_rank.to_excel(writer, sheet_name='translation',header = True,index=False)
i_r_rank.to_excel(writer, sheet_name='international retuns',header = True,index=False)
fba_rank.to_excel(writer, sheet_name='fba preparation',header = True,index=False)
stor_rank.to_excel(writer, sheet_name='storage',header = True,index=False)
ipa_rank.to_excel(writer, sheet_name='ip accelarator',header = True,index=False)
exec_rank.to_excel(writer, sheet_name='excess inventory',header = True,index=False)
cat_rank.to_excel(writer, sheet_name='cataloguing',header = True,index=False)
tax_rank.to_excel(writer, sheet_name='taxes',header = True,index=False)
comp_rank.to_excel(writer, sheet_name='compliance',header = True,index=False)
adz_rank.to_excel(writer, sheet_name='ad optimization',header = True,index=False)
am_rank.to_excel(writer, sheet_name='account management',header = True,index=False)
ebc_rank.to_excel(writer, sheet_name='ebc',header = True,index=False)
snr_rank.to_excel(writer, sheet_name='snr',header = True,index=False)
ds_rank.to_excel(writer, sheet_name='ds',header = True,index=False)
writer.save()

C:\Users\arvinmvn\AppData\Local\Temp\ipykernel_16672\402810154.py:26: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
